# Getting Started with FFTBoost

This notebook provides a quick, hands-on introduction to the `fftboost` library. Our goal is to walk through a complete, end-to-end evaluation: from configuration to running the model to interpreting the final results.

By the end of this notebook, you will know how to:
1.  Configure an `fftboost` evaluation.
2.  Run the full cross-validation harness using the Python API.
3.  Understand the statistical acceptance gates that measure success.
4.  Run the same evaluation using the Command Line Interface (CLI).

## Setup: Installation and Imports

First, let's install the library directly from GitHub and import the necessary components.

In [ ]:
# If running in a new environment, uncomment and run the line below
# !pip install git+https://github.com/pinballsurgeon/fftboost.git

from fftboost.api import FFTBoost, load_config_from_yaml
import yaml
import numpy as np
import json

## Step 1: Configuration

Every `fftboost` run is controlled by a YAML configuration file. This ensures that experiments are reproducible. For this demo, we'll use a smaller, faster version of the default configuration. The `%%writefile` magic command will create this file for us in the current directory.

In [ ]:
%%writefile demo_config.yaml

seed: 1337
fs: 1000
duration_s: 20  # Shorter duration for a quick run
window_s: 0.4
hop_s: 0.2
cv_folds: 3     # Fewer folds for speed

use_wavelets: true
wavelet_family: "db4"
wavelet_level: 4
use_hilbert_phase: true
coherence_subbands:
  - [1, 40]
  - [40, 100]

target:
  ehi_weights: { thd: 0.45, ipr: 0.55 }

fftboost:
  atoms: 5
  lambda_hf: 0.10
  lambda_coh: 3.0
  min_sep_bins: 3
  ridge_alpha: 0.1

## Step 2: Run the Evaluation with the Python API

Now we'll use our `FFTBoost` class to run the evaluation. We load the configuration, initialize the class, and call the evaluation method.

In [ ]:
print("Loading configuration...")
config = load_config_from_yaml("demo_config.yaml")

print("Initializing FFTBoost model...")
model = FFTBoost(config)

print("Running cross-validation... (This may take a moment)")
model.run_evaluation_with_generator()

print("Evaluation complete!")

## Step 3: Interpret the Results

The most important output is the **J-Beats Acceptance Gate**. This gate tells us if our `fftboost` model is statistically significantly better than the simple baseline model. Let's retrieve and examine the telemetry.

In [ ]:
# We need our NumpyEncoder, just like in the CLI, to handle numpy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.bool_):
            return bool(obj)
        return super().default(obj)

j_beats_telemetry = model.get_j_beats_telemetry()

print(json.dumps(j_beats_telemetry, indent=2, cls=NumpyEncoder))

### Understanding the J-Beats Gate

- **`j_beats_pass`**: The final verdict. `true` means we have a statistically significant win.
- **`mean_delta`**: The average improvement in R² score that `fftboost` provided over the baseline across all CV folds. A positive number is better.
- **`ci_low`** and **`ci_high`**: The 95% confidence interval for the `mean_delta`. This is the most important part.

The rule for passing the gate is simple: **`ci_low` must be greater than 0.**

This is a high standard. It means that we are 95% confident that, even in a worst-case scenario, the FFTBoost model still provides a positive improvement. This is how we make decisions based on data, not luck.

## Bonus: Running with the CLI

You can achieve the exact same result directly from your terminal using the `fftboost` command. The `!` symbol lets us run shell commands from inside the notebook.

In [ ]:
!fftboost --config demo_config.yaml

## Next Steps

This notebook covered the basic evaluation workflow. For more advanced configuration options and details on the underlying algorithms, please refer to the project's `README.md` and the documents in the `docs/` folder.